## Figure 4 - affect of learning rates on user performance and encoder

In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import wilcoxon as wilcoxon


# meta analysis functions
import sys
sys.path.append('/code/')
from util import analysis
from util import plotting
from util import util_continuous as utils

In [ ]:
PATH = '/data/'

In [ ]:
with open(PATH + 'time-domain-error/time-domain-error-30sec-in-cm.pkl','rb') as handle:
    td_error, td_error_first, td_error_last, t0_start, t0_end, t1_end, td_diff, td_diff_slow, td_diff_fast, td_diff_pos, td_diff_neg, td_diff_pD3, td_diff_pD4 = pickle.load(handle)

with open(PATH + 'encoder-estimation-data/encoder-decoder-data.pickle', 'rb') as handle:
    encoder, encoder_r2, idx_dict, pos_vel_model, pos, dec_vels, decoders = pickle.load(handle)
keys = ['METACPHS_S106', 'METACPHS_S107','METACPHS_S108', 'METACPHS_S109', 'METACPHS_S110', 'METACPHS_S111', 'METACPHS_S112', 'METACPHS_S113', 'METACPHS_S114', 'METACPHS_S115', 'METACPHS_S116', 'METACPHS_S117', 'METACPHS_S118', 'METACPHS_S119']




Table of variables used 

| Variable | Shape | Shape Representations | Meaning |
| --- | --- | --- | --- |
| td_error | (2, 14, 8, 20770) | blocks x subjects x conditions x time points | this is the time domain error (Euclidean Distance) between the cursor and target at each time point

Explanations of shapes:
* number of blocks = 2 -- experimental setup
* numebr of subject = 14 -- experimental setup
* number of conditions = 8 -- experimental setup of decoder conditions

In [ ]:
assert(td_error.shape == (utils.n_blocks, utils.n_keys, utils.n_conds, utils.min_time))

In [ ]:
# Import seaborn
import seaborn as sns

sns.set_theme(style="ticks", rc=utils.sns_custom_params, font_scale=0.6)


In [ ]:
label_size = 6
## SETUP THE FIGURE HERE
## HAVE TO RE-REUN FROM HERE TO "CLEAR" THE PLOT
fig_error_rate = plt.figure(figsize = (6.3, 2), layout='constrained') # set the total figure size
mosaic = """
    aabc
    """

# set up the axes
ax_dict = fig_error_rate.subplot_mosaic(mosaic)
for ii in ax_dict:
    plotting.remove_and_set_axes(ax_dict[ii], bottom=True, left=True)
    ax_dict[ii].tick_params(axis='both', which='major', labelsize = label_size)
    ax_dict[ii].tick_params(axis='both', which='minor', labelsize = label_size)
fig_error_rate.patch.set_facecolor('white')

# a
# time-domain error 
ax_dict['a'].set_ylabel("Mean Error (cm)")
ax_dict['a'].set_xlabel("Time (minutes)")
ax_dict['a'].set_title("Time-Domain Error - Learning Rates")

# b 
ax_dict['b'].set_ylabel("% Change in Error")

# b 
ax_dict['c'].set_ylabel(r'$|E_{f} - E_{i}|$')


fig_error_rate.patch.set_facecolor('white')


In [ ]:
# axis a
ax = ax_dict['a']

data_fast = np.mean(td_error[:, :, utils.fast, utils.RAMP:], axis=(0, 2)) # (2, 7, 4, 20770)
data_slow = np.mean(td_error[:, :, utils.slow, utils.RAMP:], axis=(0, 2)) # (2, 7, 4, 20770)

print("N = ", data_fast.shape)

# number of samples per seconds
trial_time = 300 # 300 second trials
tscale = int(utils.min_time/trial_time) # number of samples per seconds
time_x = np.linspace(0, 5, utils.min_time) # time in minutes
ks = int(5*tscale) # set kernal size for smoothing to be in seconds

# # check to make sure that we're comparing N of keys across the 2 conditions
axis = 0
assert(data_fast.shape[0] == utils.n_keys)
assert(data_slow.shape[0] == utils.n_keys)

plotting.plot_smooth_time_domain(time_x[utils.RAMP:], data_fast, utils.n_keys, ls='--', axis = axis, 
                                 kernal_size = ks, ax=ax, color=utils.colors['fast'], alpha=0.4,
                                 lw=1.5, label='fast', remove_axes = False)
plotting.plot_smooth_time_domain(time_x[utils.RAMP:], data_slow,  utils.n_keys, axis = axis, 
                                 kernal_size = ks, ax=ax, color=utils.colors['slow'],  alpha=0.4,
                                 lw=1.5, label='slow', remove_axes = False)

# ax.hlines(y=2.5, xmin = 0, xmax = 5, ls = '--', lw = 0.5, color  = 'black')

# ax.set_ylim(0, 45)
fig_error_rate

In [ ]:
td_early = td_error[:, : ]

In [ ]:
# fig b - performance across learning rates

axs = ax_dict['b']

data1 = np.ndarray.flatten(td_diff_slow)
data2 = np.ndarray.flatten(td_diff_fast)

# data1 = np.median(td_error[:, :, utils.fast, :(70*20)], axis = (0, 2, 3))
# data2 = np.median(td_error[:, :, utils.slow, :(70*20)], axis = (0, 2, 3))
assert(data1.shape == data2.shape == (utils.n_keys,))

data_groups = [data1, data2] # slow, fast
data_labels = ['Slow', 'Fast']
data_pos = [0, 0.5]
bplot = axs.boxplot(data_groups, 
                    showfliers=False,
                    patch_artist=True,
                    positions=data_pos,
                    widths = 0.2,
                    boxprops=dict(edgecolor="none"),
                    medianprops=dict(color='black', lw=1))



for patch, color in zip(bplot['boxes'], [utils.colors['slow'], utils.colors['fast']]):
    patch.set_facecolor(color)

# # rotate labels  
axs.set_xticks(data_pos, data_labels, rotation=40)

w = wilcoxon(data1, data2) 
print(w)

plotting.plot_significance(pvalue = w.pvalue, data1=data1, data2 = data2, 
                           data_pos = data_pos, fig=fig_error_rate, 
                           ax=axs, fontsize=10, lw=1, y_asterix=5, y_bar=3)

fig_error_rate

In [ ]:
## set up encoder changes

## this code sets up the encoder variables to be plotted and compared in this notebook

## set up variables to use for this code
# number of features of the encoder
# 8 features of the encoder = target position (x, y), target velocity (x, y), position error (x, y), velocity error (x, y)
n_feat = 8 

## take the encoder variables without the affine term 
# this is done because the affine term was to fit the encoder, but does not represent any encoding of the user
encoder_linear = encoder[:, :, :, :, :, :-1]
# check that encoder_linear variable is the right shape -- same as the encoder, but the last axis should be the number of features
assert(encoder_linear.shape == (utils.n_blocks, utils.n_keys, utils.n_conds, 
                                len(utils.update_ix) - 2, utils.n_ch, n_feat))

## average the encoder across minute to use the minute-averaged encoder as a measurement of the user
# this averaging is done so that we can reduce the overfitting of the encoder to the 20-second intervals
axis_time = 3 # the time axis is the 4th one, so = 3
enc_all_min1 = np.mean(encoder_linear[:, :, :, 1:4], axis = axis_time) # average across the first minute
enc_all_min2 = np.mean(encoder_linear[:, :, :, 4:7], axis = axis_time) # average across the 2nd minute
enc_all_min3 = np.mean(encoder_linear[:, :, :, 7:10], axis = axis_time) # average across the 3rd minute
enc_all_min4 = np.mean(encoder_linear[:, :, :, 10:13], axis = axis_time) # average across the 4th minute
enc_all_min5 = np.mean(encoder_linear[:, :, :, -3:], axis = axis_time) # average across the last minute
assert(enc_all_min1.shape == enc_all_min2.shape == enc_all_min3.shape 
       == enc_all_min4.shape == enc_all_min5.shape 
       == (utils.n_blocks, utils.n_keys, utils.n_conds, utils.n_ch, n_feat))

# stack the minute-averaged encoders together so we can find the change in the users' encoders minute to minute
enc_all_min = np.stack((enc_all_min1, enc_all_min2, enc_all_min3, enc_all_min4, enc_all_min5))
# 5 in the first axis since we're stacking 5 arrays together
assert(enc_all_min.shape == (5, utils.n_blocks, utils.n_keys, utils.n_conds, utils.n_ch,  n_feat))

## subtract the difference in the encoder from minute to minute and then take the frobenious norm of that difference
# this is |E_t+1 - E_t|_F
enc_all_diff = np.linalg.norm(np.diff(enc_all_min, axis = 0), axis = (-1, -2))

# this is |E_final - E_initial|_F
enc_all_fi = np.linalg.norm(enc_all_min5 - enc_all_min1, axis = (-1, -2))
# shape is subjects x trials 
assert(enc_all_fi.shape == (utils.n_blocks, utils.n_keys, utils.n_conds))

# find the mean difference per subject
enc_all_diff_subj = np.mean(enc_all_diff, axis = (0, 1, 3))
enc_fi_diff_subj = np.mean(enc_all_fi, axis = (0, 2))
assert(enc_all_diff_subj.shape == enc_fi_diff_subj.shape == (utils.n_keys, ))


In [ ]:
# get one |E_f - E_i\ per condition per subject

# SLOW, FAST - mean across subjects
enc_fi_diff_fast = np.mean(enc_all_fi[:, :, utils.fast], axis = (0, 2))
enc_fi_diff_slow = np.mean(enc_all_fi[:, :, utils.slow], axis = (0, 2))

# POS, NEG - mean across subjects
enc_fi_diff_pos = np.mean(enc_all_fi[:, :, utils.pos_init], axis = (0, 2))
enc_fi_diff_neg = np.mean(enc_all_fi[:, :, utils.neg_init], axis = (0, 2))

# LAMBDA HIGH VS LAMBDA LOW- median across subjects
enc_fi_diff_pD4 = np.mean(enc_all_fi[:, :, utils.pD_3], axis = (0, 2))
enc_fi_diff_pD3 = np.mean(enc_all_fi[:, :, utils.pD_4], axis = (0, 2))

d_fast = np.ndarray.flatten(enc_fi_diff_fast)
d_slow = np.ndarray.flatten(enc_fi_diff_slow)
d_pos = np.ndarray.flatten(enc_fi_diff_pos)
d_neg = np.ndarray.flatten(enc_fi_diff_neg)
d_pd4 = np.ndarray.flatten(enc_fi_diff_pD4) # low penalty term
d_pd3 = np.ndarray.flatten(enc_fi_diff_pD3) # high penalty term

# check for shapes across comparison
assert(len(d_fast) == len(keys))
assert(len(d_slow) == len(keys))
assert(len(d_pos) == len(keys))
assert(len(d_neg) == len(keys))
assert(len(d_pd4) == len(keys))
assert(len(d_pd3) == len(keys))

In [ ]:
## add subfigure to the combined plots

axs = ax_dict['c']

data_groups = [d_slow, d_fast]
data_labels = ['slow', 'fast']
data_pos = [0, 0.4]
bplot = axs.boxplot(data_groups, 
                    showfliers=False,
                    patch_artist=True,
                    positions=data_pos,
                    widths=0.2,
                    boxprops=dict(edgecolor="none"),
                    medianprops=dict(color='k', lw=1))


for patch, color in zip(bplot['boxes'], 
                               [utils.colors['slow'], utils.colors['fast']]):
            patch.set_facecolor(color)
        
# rotate labels  
axs.set_xticks(data_pos,data_labels, rotation=40)

# set labels and axes
# axs.tick_params(axis='x', labelsize=tck_size)
# learning rate
w1 = wilcoxon(np.ndarray.flatten(d_slow), np.ndarray.flatten(d_fast))
print("learning rate, N = ", d_fast.shape)
pv1 = w1.pvalue
print(w1)

w2 = wilcoxon(np.ndarray.flatten(d_pos), np.ndarray.flatten(d_neg))
print("init, N = ", d_pos.shape)
pv2 = w2.pvalue
print(w2)

w3 = wilcoxon(np.ndarray.flatten(d_pd4), np.ndarray.flatten(d_pd3))
print("penalty, N = ", d_pd4.shape)
pv3 = w3.pvalue
print(w3)



axs.set(ylabel = '$|E_{f} - E_{i}|$')

plotting.plot_significance(pvalue = pv1, data1=d_slow, data2 = d_fast, 
                           data_pos = data_pos, fig=fig_error_rate, 
                           ax=axs, fontsize=10, lw=1, y_asterix=3.5, y_bar=3)

fig_error_rate

In [ ]:
image_format = 'pdf' # e.g .png, .svg, etc.
image_name = 'fig4-learning-rates-mean-cm.pdf'
PATH = '/results/'
fig_error_rate.savefig(PATH + image_name, format=image_format, dpi=300)